# ADATOK FORRÁSA ÉS BETÖLTÉSE PYTHONBA

Az adatok Eurostat-ról származnak. Részletes leírása az `eurostat` könyvtárnak:
https://pypi.org/project/eurostat/
<br>
Első lépés - instalálni a könyvtárat:

In [ ]:
#!python -m pip install eurostat

In [ ]:
import eurostat
import pandas as pd

`read a dataset from the main database` fejezet
<br> Az én dataset-em amit olvasni akarok: _sbs_na_ind_r2_

In [ ]:
df = eurostat.get_data_df('sbs_na_ind_r2')
df

# ADATFORMÁZÁS

## A lekért táblázat formázása

Mivel csak 5 évre visszamenőleg akarom ábrázolni, 2014-től arra eldobom az adatokat

In [ ]:
for year in range(2005,2015,1):
    df.drop(year, axis=1, inplace=True)

Átnevezem az oszlopokat

In [ ]:
df = df.rename(columns={'indic_sb': 'indicator', 'geo\\time': 'country_code'})
df

Csak európai országokat szeretnék, ezért a többit ugyancsak eldobom. Ugyanúgy az Uniós összesített adatokat is meg Liechtenstein-t ahol nem voltak adatok

In [ ]:
df["country_code"].unique()

In [ ]:
df = df.set_index('country_code').drop(['EU27_2007', 'EU27_2020', 'EU28', 'TR', 'TOTAL', 'LI'],axis=0).reset_index()

## A mutatók kiválasztása és az `indicator` kódok leírása

Az eurostat könyvtár segítségével le lehet kérni a kódokat: get an Eurostat dictionary => `eurostat.get_dic(code)`

_A NACE kódokat is le lehetne innen tölteni, de nekem meg magyarul kellenének._


Az összes mutató letöltése ami megtalálható az eurostat-on

In [ ]:
indicators = eurostat.get_dic('indic_sb')
indicators

Azon mutatók kiszűrése és magyar megfelelőik amelyek engem érdekelnek

In [ ]:
needed_indicators = {
    'Enterprises - number': 'Vállalatok száma',
    'Value added at factor cost - million euro': 'Hozzáadott érték tényezőköltségen - millió euró',
    'Turnover or gross premiums written - million euro': 'Árbevétel vagy bruttó díjbevétel - millió euró',
    'Wages and Salaries - million euro': 'Személyzeti kiadások - millió euró',
    'Total purchases of goods and services - million euro': 'Termék és szolgáltatás vásárlások összesen - millió euró',
    'Gross operating surplus - million euro': 'Bruttó működési eredmény - millió euró',
}

needed_indicator_codes = list(filter(lambda key: indicators.get(key) in needed_indicators.keys(), indicators.keys()))

needed_indicator_codes #tartalmazza a mutatók kódját amelyek nekem kelleni fognak


Csak azokat az indicator-okat hagyom meg a df-ben, amelyek a `needed_indicator_codes` között van

In [ ]:
df = df[df.indicator.isin(needed_indicator_codes)]
df

A mutatók magyar leírásának beszúrása

In [ ]:
df.loc[:, "indicators_desc_hu"] = df.indicator.map(lambda code: needed_indicators[indicators[code]])
df

Mivel a vizualizációban a személyzeti kiadások és a termék és szolgáltatás vásárlások összege fog kelleni, ezeket nincs értelme külön tárolnom a df-mben, hanem elég lenne egy kiadások összesen mutatót használni

In [ ]:
#kimentem ezt a két változót
wage_exp = df.loc[df["indicators_desc_hu"] == "Személyzeti kiadások - millió euró"]
goods_exp = df.loc[df["indicators_desc_hu"] == "Termék és szolgáltatás vásárlások összesen - millió euró"]

#törlöm ezeket a df-ből
df.drop(df.loc[df["indicators_desc_hu"].isin(["Személyzeti kiadások - millió euró", "Termék és szolgáltatás vásárlások összesen - millió euró"])].index, inplace=True)

Azt a kiadás tömböt járom be amelyikben kevesebb adat van, hogy ne tömjem meg a df-em nan-okkal

In [ ]:
print(len(wage_exp)) 
print(len(goods_exp))


Mindkét tömbnek beállítom indexnek az országot és a NACE kódot, hiszen így végig tudom majd járni az egyik kulcsait és egy `loc` segítségével megkapom a másikból az értéket.

In [ ]:
wage_exp.set_index(["country_code", "nace_r2"], inplace=True)
goods_exp.set_index(["country_code", "nace_r2"], inplace=True)

Felépítek egy dictonary-t amelyik az összegeket fogja tartalmazni

In [ ]:
import numpy as np

exp_dict = {}
row = 0
for i in goods_exp.index:

    country_dict = {}

    country_dict["country_code"] = list(i)[0];
    country_dict["nace_r2"] = list(i)[1];
    country_dict["indicator"] = 'EXP';

    for year in range(2019, 2014, -1):
        if np.isnan(wage_exp.loc[i, year]) | np.isnan(goods_exp.loc[i, year]): #ha bármelyik NaN, legyen NaN érték, ne adja össze csak a nem-NaN értéket
            country_dict[year] = np.nan;
        else:
            country_dict[year] = (wage_exp.loc[i, year] + goods_exp.loc[i, year])

    country_dict["indicators_desc_hu"] = 'Termék és szolgáltatás vásárlások és személyzeti kiadások összesen';

    exp_dict[row] = country_dict;
    row += 1;

exp_dict


A kapott dictionary-t pedig átalakítom pd.DataFram-é

In [ ]:
#mivel sokkal gyorsabb az átalakítás ha kevés sor van, nem specifikálom, hogy az index a dict.value legyen, hanem utólag transponálom

df = pd.concat([df, pd.DataFrame(exp_dict).T], ignore_index=True)
df

## a NACE kódok leírása

Eurostatról is le lehetne kérni az adatot, viszont azok az angol elnevezését adnák vissza a NACE kódoknak, nekem pedig a magyar kell.
A <a href="https://www.ksh.hu/teaor_menu">KSH oldalán</a> megtaláltam .xls formátumban a táblázatot a NACE kódokkal.

In [ ]:
# !python -m pip install xlrd

Beolvasom Excelből az adatot és az első két oszlopon kívül eldobom az összeset

In [ ]:
naces = pd.read_excel("NaceCodes_hu.xls", header=1)
naces.drop(naces.columns[2:6], axis=1, inplace=True)
naces.columns = ["code", "desc_hu"]
naces.drop(naces.loc[(naces["code"].isna()) | (naces["desc_hu"].isna())].index, inplace=True)
naces

A kódok az én df-ben B-től E-ig mennek => a többi betűt el kell dobni és minden kód elé be kell tenni a NACE kategória betűjét

In [ ]:
#mivel van ahol int és van ahol str a kód, átalakítom az egész oszlopot stringgé
naces["code"] = naces["code"].astype(str)

#felépítem, hogy melyik indexeknél változik meg a NACE betűje
nace_cat_at_index = {}
for i in naces.index:
    if (len(naces["code"][i]) == 1):
        nace_cat_at_index[naces["code"][i]] = i 

In [ ]:
nace_cat_at_index

In [ ]:
#mindig megyek két betű közötti range közt és ha B-E kategóriában van, akkor a számok elé illesztem a betűket, különben törlöm a range-t

max_range = len(naces.index) + 1

for i in reversed(nace_cat_at_index):
    if ord(i) not in range(ord('B'), ord('E') + 1):
        naces.drop(naces.iloc[(nace_cat_at_index[i]):max_range - 1].index, inplace=True)
    else:
        naces.loc[(nace_cat_at_index[i] + 1):max_range - 1, "code_w_letter"] = i + naces.loc[(nace_cat_at_index[i] + 1):max_range - 1, "code"]
    max_range = nace_cat_at_index[i]

#beállítom a betű NACE kategóriákat is
naces.loc[naces["code_w_letter"].isna(), "code_w_letter"] = naces.loc[naces["code_w_letter"].isna(), "code"]

Maradnak felesleges space-k a kódokban => trimmelni kell a code_w_letter oszlopot

In [ ]:
naces.code_w_letter = naces.code_w_letter.map(lambda code: code.strip())

Beteszem a magyar NACE leírásokat a df-be

In [ ]:
naces.set_index("code_w_letter", inplace=True)
df.loc[:, "nace_r2_desc_hu"] = df.nace_r2.map(lambda nace: naces.loc[nace, "desc_hu"])
df

## Az országok nevének beszúrása

<a href= "https://hu.wikipedia.org/wiki/ISO_3166-1">Wikipédiáról</a> kinyerem az országok magyar elnevezését és az ISO2 kódokat.

In [ ]:
country_iso = pd.read_html('https://hu.wikipedia.org/wiki/ISO_3166-1')[1]
country_iso.rename(columns={"Ország / Régió": "country", "Alpha-2": "ISO2"}, inplace=True)

#a megkapott df-ből csak az ország és Alpha-2 oszlopok kellenek
country_iso.drop(['Numerikus', 'Alpha-3', 'Helyi ISO kódok'], axis=1, inplace=True)
country_iso

Az országok magyar elnevezésének beszúrása

In [ ]:
country_iso.set_index("ISO2", inplace=True)


In [ ]:
#amikor lefuttattam a map()-et EL kulcsot nem talált. Ekkor lekérdeztem az Eurostat API-jával, hogy az EL kód melyik országnak felel meg
# eurostat.get_dic("geo")["EL"] # ---> Görögország

#a Wikipédián és a topojson file-omban Görögország rövidítése GR volt, így a df-emben kicseréltem az EL-t GR-re.
# country_iso[country_iso.country == "Görögország"]

## ugyanez volt 'UK' => 'GB'-vel is
# eurostat.get_dic("geo")["UK"] # ---> United Kingdom 
# country_iso[country_iso.country == "Egyesült Királyság"]

df.country_code.replace('EL', 'GR', inplace=True)
df.country_code.replace('UK', 'GB', inplace=True)

df["country_name"] = df.country_code.map(lambda code: country_iso.loc[code][0]) #.loc[code][0], mert .loc[code] visszatérít egy Series-t
df


## GDP értékek hozzárendelése

GDP értékek lekérése eurostatról

In [ ]:
gdp_df = eurostat.get_data_df('nama_10_gdp')
gdp_df

Ugyancsak 2015-2019 évekre kellenek az adatok, ezért a többit eldobom

In [ ]:
for i in range(1975, 2015, 1):
    gdp_df.drop(i, axis=1, inplace=True)
gdp_df.drop(2020, axis=1, inplace=True)
gdp_df

A dataset-ből nekem csak a `Value added, gross` mutató kell 

In [ ]:
gdp_indicators = eurostat.get_dic("na_item")

val_added = list(filter( lambda ind: gdp_indicators[ind] == 'Value added, gross' ,gdp_indicators.keys()))

gdp_df = gdp_df[gdp_df["na_item"].isin(val_added)]
gdp_df

Ugyancsak a `Current prices, million euro` unit kell

In [ ]:
units = eurostat.get_dic("unit")
curr_prices = list(filter(lambda key: units[key] == "Current prices, million euro" ,units.keys()))

gdp_df = gdp_df[gdp_df["unit"].isin(curr_prices)]
gdp_df = gdp_df.rename(columns={"geo\\time": "country"})
gdp_df

Mielőtt összevonnánk a két táblát, ügyelni kell az előbb azonosított két hibás országkódra

In [ ]:
gdp_df["country"].replace('EL', 'GR', inplace=True)
gdp_df["country"].replace('UK', 'GB', inplace=True)

Eldobom azokat az országokat (sorokat) amelyek nincsenek benne az én df-ben

In [ ]:
gdp_df = gdp_df.loc[gdp_df["country"].isin(df.country_code.unique())]

Ezután olyan formára kell hozzuk mint amilyen a df-em, hogy tudjam concat-olni a két adatot

In [ ]:
#unit, na_item oszlopokra nincs szükség
gdp_df.drop(["unit", "na_item"], axis=1, inplace=True)

# country átnevezése
gdp_df = gdp_df.rename(columns= {"country": "country_code"})

#az indicator beszúrása
gdp_df.insert(0, "indicator", ["GDP" for i in range (gdp_df["country_code"].size)])
gdp_df.insert(2, "nace_r2", ["GDP" for i in range (gdp_df["country_code"].size)])

# hozzáadjuk az indicator angol és magyar leírásait
gdp_df.loc[:, "indicators_desc_hu"] = "Bruttó hozzáadott érték - folyó áron"
gdp_df.loc[:, "nace_r2_desc_hu"] = "Bruttó hozzáadott érték - folyó áron"

# country_name oszlop hozzáadása
gdp_df.loc[:, "country_name"] =gdp_df.country_code.map(lambda code: country_iso.loc[code][0])

gdp_df

Végül pedig összefűzöm a `df` és `gdp_df` dataframeket

In [ ]:
df = pd.concat([df, gdp_df], ignore_index=True)
df

# NaN ÉRTÉKEK ELVETÉSE

Ne legyen olyan sor ahol az összes érték NaN

In [ ]:
df.drop(df.loc[df[2019].isna()].loc[df[2018].isna()].loc[df[2017].isna()].loc[df[2016].isna()].loc[df[2015].isna()].index, axis=0, inplace=True)
df

# SOR ALAPÚ FORMÁTUMRA HOZÁS

In [ ]:
df = df.set_index(["indicator", "country_code", "nace_r2", "indicators_desc_hu", "nace_r2_desc_hu" ,"country_name"]).stack().reset_index()

In [ ]:
df.columns=["indicator", "country_code", "nace_r2", "indicators_desc_hu", "nace_r2_desc_hu" , "country_name", "year", "value"]

In [ ]:
df

# ADATOK EXPORTÁLÁSA

In [ ]:
df.to_csv('data.csv', index=False)